In [27]:
import jax
import jax.numpy as jnp
from enum import IntEnum
from memo import memo

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd


Auction valuation (making an estimate, taking emotions into account – plausible deniability) – Part II
Now, imagine that you’re not buying the painting for yourself. Rather, you’re a valuations manager for an auction house.

> Your job is now to tell the seller (your friend) how much you’d be willing to pay for the painting, given how much you think it’ll go for at auction tomorrow. You might model this as a simple point estimate (the painting is worth $10,000) or as a range, like a Gaussian, where you tell them the mean and variance (I think it’ll go for \$10,000, but +/- \$1K). 

Now, you might imagine taking into account at least two competing factors, both involving how your friend might feel:
How much your friend thinks that you value their painting (as before), relative to how much they think its worth;
How your friend will actually feel when the painting sells tomorrow. This could actually have two components:
How surprised / disappointed your friend is by the actual amount they make;
How surprised / disappointed your friend feels by how this sale reflects on their actual, internal estimate of the painting. This is where the ‘plausible deniability’ aspect comes into account – if you tell them a very wide range, its possible that (a) you think that the painting could be worth a lot, but (b) they could just seem to have a ‘fluke’ bad day tomorrow – saving face as to how much the painting really was worth.


In [ ]:
#class A(IntEnum): Low=0; Medium=1; High=2
#class Friend(IntEnum): Indiferent=0; Friend=1

class Competence(IntEnum): Low=0; High=1
class Quality(IntEnum): Low=0; High=1

offers = 100*jnp.linspace(1, 5, num=5)

@jax.jit
def price_given_quality(q):
    return jnp.array([200, 400])[q]

@jax.jit
def uncertainty_given_competence(c):
    return 100*jnp.array([1.0, .5])[c]

@jax.jit
def offer_given_competence_quality(o, c, q):
    return jax.scipy.stats.norm.pdf(o, loc=price_given_quality(q), scale=uncertainty_given_competence(c))


In [173]:
# type: ignore

@jax.jit
def get_beta_prior(q, case):
    return jnp.array([
        [1, 1],  # Unsure
        [4, 1],  # Low competence / Bad quality
        [1, 4],  # High competence / Good quality
    ])[case, q]

@jax.jit
def get_min(a, b):
    return jnp.minimum(a, b)

@jax.jit
def get_max(a, b):
    return jnp.maximum(a, b)

@memo
def auctioneer_makes_offer[o: offers, c: Competence, q: Quality](prior_q, prior_c, honesty, care, disappointment, elation):

    auctioneer: knows(c, q)

    auctioneer: thinks [
        painter: thinks [
            auctioneer: given(q in Quality, wpp=get_beta_prior(q, prior_q)),
            auctioneer: given(c in Competence, wpp=get_beta_prior(c, prior_c)),
            auctioneer: given(o in offers, wpp=offer_given_competence_quality(o, c, q))
        ]
    ]
    auctioneer: chooses(o in offers, wpp=exp(
                        + honesty * log(offer_given_competence_quality(o, c, q)) # Honesty / naive response
                        + imagine [
                            painter: observes [auctioneer.o] is o,
                            + care * log(painter[E[auctioneer.q]])
                            - disappointment * log(abs(get_min(1e-8, q - painter[E[auctioneer.q]])))
                            + elation * log(get_max(1e-8, q - painter[E[auctioneer.q]]))
                        ])
                    )

    return Pr[auctioneer.o == o]

honesty = 0
care = 1
disappointment = 0
elation = 0
prior_quality = 0
prior_competence = 0
print(offer)
auctioneer_makes_offer(prior_quality, prior_competence, honesty, care, disappointment, elation)[:, Competence.High, Quality.Low].round(2)


[100. 200. 300. 400. 500.]


Array([0.01      , 0.02      , 0.19999999, 0.38      , 0.39      ],      dtype=float32)

In [ ]:
# type: ignore

@jax.jit
def get_beta_prior(q, case):
    return jnp.array([
        [1, 1],  # Unsure
        [4, 1],  # Low competence / Bad quality
        [1, 4],  # High competence / Good quality
    ])[case, q]

widths = jnp.array([0.2, 1.0])

def offer_given_width_competence_quality(o, q):
    return jax.scipy.stats.norm.pdf(o, loc=price_given_quality(q), scale=.5)

def width_given_competence(w, c):
    return jnp.array([0.2, 1.0])[c]


def offer_given_competence_quality():
    return 
@memo
def auctioneer_makes_offer[o: offers, w: widths, c: Competence, q: Quality](prior_q, prior_c, honesty=1, quality_persuasion=1, competence_persuasion=1):

    auctioneer: knows(c, q)

    auctioneer: thinks [
        painter: thinks [
            auctioneer: given(q in Quality, wpp=get_beta_prior(q, prior_q)),
            auctioneer: given(c in Competence, wpp=get_beta_prior(c, prior_c)),
            auctioneer: given(o in offers, wpp=offer_given_competence_quality(o, c, q))
        ]
    ]
    auctioneer: chooses(o in offers, wpp=exp(
                        + honesty * log(offer_given_competence_quality(o, c, q)) # Honesty / naive response
                        + imagine [
                            painter: observes [auctioneer.o] is o,
                            + quality_persuasion * log(painter[Pr[auctioneer.q == {Quality.High}]])
                            + competence_persuasion * log(painter[Pr[auctioneer.c == {Competence.High}]])
                        ])
                    )

    return Pr[auctioneer.o == o]

## Who knows what?

Does the auctioneer think that:

1. the painter is certain / uncertain about the quality? Yes

If painter the is certain about the quality being high:

1. If offer is low and confident (100 $\pm$ 10) then the painter is devastated (low valence, high arousal) => Large negative update
2. If offer is low and not confident (100 $\pm$ 100) then the painter is (low valence, low arousal) => Small negative update
3. If offer is high and confident (300 $\pm$ 10) then the painter is happy (high valence, medium arousal) => Small Update
4. If offer is high and not confident (300 $\pm$ 100) then the painter is happy (high valence, low arousal) => Smaller Update